In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import pandas as pd
from scipy import stats

from src.dataset import load_data
from src.features import build_features, compare_distributions

Code from here: https://github.com/shreyashankar/debugging-ml-talk/blob/main/nyc_taxi_2020.ipynb

## Loading data

In [2]:
taxi_jan = load_data(year=2020, month=1)
taxi_feb = load_data(year=2020, month=2)
taxi_mar = load_data(year=2020, month=3)

In [3]:
len(taxi_jan), len(taxi_feb), len(taxi_mar)

(6405008, 6299367, 3007687)

## Pre-processing

In [4]:
target_col = "high_tip"

taxi_jan = build_features(taxi_jan.sample(100000), target_col)
taxi_feb = build_features(taxi_feb.sample(100000), target_col)
taxi_mar = build_features(taxi_mar.sample(100000), target_col)

## Test distributions

Maybe the distribution of data shifted. We could try to quantify this using a 2-sided statistical test (Kolmogorov Smirnov in this example).

### Compare January 2020 vs February 2020

The p values being small doesn't really tell us much, as we get very small p values when comparing January 2020 vs February 2020 even though we know the F1 score was similar. Curse "big data."

In [6]:
comparison_df = compare_distributions(taxi_jan, taxi_feb)
comparison_df.sort_values(by='p_value', ascending=True).head(11)

,feature,statistic,p_value
0,pickup_weekday,0.047419,8.238052e-98
7,trip_speed,0.039734,8.571128e-69
2,work_hours,0.027527,3.099905e-33
6,trip_time,0.017964,2.122676e-14
1,pickup_hour,0.008107,2.845354e-03
5,trip_distance,0.006727,2.189595e-02
8,PULocationID,0.005767,7.238714e-02
9,DOLocationID,0.004849,1.912818e-01
3,pickup_minute,0.003681,5.082157e-01
4,passenger_count,0.002981,7.664661e-01


### Compare February 2020 vs March 2020

These p values are also small, which is good? But if this method in general sends warning alerts all the time, an end user might not trust it.

In [7]:
comparison_df = compare_distributions(taxi_feb, taxi_mar)
comparison_df.sort_values(by='p_value', ascending=True).head(11)

,feature,statistic,p_value
7,trip_speed,0.065278,5.573488e-185
0,pickup_weekday,0.063414,1.394967e-174
2,work_hours,0.039483,6.602832e-68
1,pickup_hour,0.024527,1.834635e-26
6,trip_time,0.021801,5.353301e-21
5,trip_distance,0.021554,1.561496e-20
4,passenger_count,0.020344,2.435018e-18
9,DOLocationID,0.016979,6.649330e-13
8,PULocationID,0.014496,1.602196e-09
10,RatecodeID,0.005068,1.541831e-01
